In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer
%matplotlib inline

In [4]:
data = pd.read_csv('../Data/Amazon.csv')
# data = data[0:1000]
data.shape
# data.head(5)

(455000, 13)

In [5]:
# define y
y = data.iloc[:, 12].values
y.shape

(455000,)

In [6]:
#bag of words on text // Replace with hashingvectorizer
# vectorize Bag of Words from review text; as sparse matrix
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=2 ** 17, non_negative=True)
XText = hv.transform(data.Text)
XText.shape
# textArray = X.as_matrix()
# type(X)
# features = count.fit_transform(textArray)
# XText = features.toarray()

(455000, 131072)

In [7]:
#bag of words on summary
# creating newSummary with "no summary given" where summary does not exist.
# data['SummaryFill'] = np.where(pd.isnull(data['Summary']) == True, 'no summary given', data['Summary'])
# from sklearn.feature_extraction.text import CountVectorizer
# count = CountVectorizer()
data['Summary'].fillna('null', inplace=True)
XSummary = hv.transform(data.Summary)
XSummary.shape
# print(type(data.Summary))
# summaryArray = data.Summary.as_matrix()
# type(summaryArray)
# features = count.fit_transform(summaryArray)
# XSummary = features.toarray()
# XSummary.shape

(455000, 131072)

In [8]:
# features from Amazon.csv to add to feature set
data['reviewLen'] = data['Text'].str.len()
data['summaryLen'] = data['Summary'].str.len()
data['usernameWords'] = data ['ProfileName'].str.split().str.len()

XScore = data.iloc[:, 7].values.reshape(data.shape[0], 1)
XreviewLen = data.iloc[:, 13].values.reshape(data.shape[0], 1)
XsummaryLen = data.iloc[:, 14].values.reshape(data.shape[0], 1)
XusernameWords =data.iloc[:, 15].values.reshape(data.shape[0], 1)

Xtoadd = np.concatenate((XScore,XreviewLen,XsummaryLen,XusernameWords), axis=1)


In [9]:
# vectorize Bag of Words from Summary text; as sparse matrix
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=2 ** 17, non_negative=True)
XhashText = hv.transform(data.Text)
XhashSummary = hv.transform(data.Summary)

In [10]:
#Fix missing values in summary
# data["Summary"].fillna(data["Summary"].mean(), inplace=True)
# data["Summary"].fillna('pad')
# data["Summary"].fillna('pad')
# data.Summary.fillna(0)
# data['newSummary'] = np.where(pd.isnull(data['Summary']) == True, 'no summary given', data['Summary'])
# Y = hv.transform(data.Summary)

# imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
# imp.fit(hv.transform(data.Summary))

In [11]:
# convert to CSR
from scipy.sparse import csr_matrix, hstack
XtoaddSparse = csr_matrix(Xtoadd)
Xfinal = hstack([Xtoadd, XtoaddSparse,XhashText,XhashSummary,XText,XSummary])
X = csr_matrix(Xfinal)
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X = imp.fit_transform(X)

In [12]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(318500, 524296)
(318500,)
(136500, 524296)
(136500,)


In [13]:
# report on training and test sets
def print_results():
    print('Error rate on training set: ')
    print((y_train != y_pred).sum() / X_train.shape[0])
    print('Accuracy rate on training set: ')
    print(1 - (y_train != y_pred).sum() / X_train.shape[0])
    print('True positive rate on training tet:')
    print(((y_train==True) & (y_pred==True)).sum() / y_train.sum())
    print('**************')
    print('Error rate on test set: ')
    print((y_test != y_pred_test).sum() / X_test.shape[0])
    print('Accuracy rate on test set: ')
    print(1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    print('True positive rate on test set')
    print(((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    print('True negative rate on test set')
    print(((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))

In [14]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [15]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1.0, random_state=0)
y_pred = lr.fit(X_train, y_train).predict(X_train)
y_pred_test = lr.predict(X_test)

In [16]:
print_results()

Error rate on training set: 
0.0719120879121
Accuracy rate on training set: 
0.928087912088
True positive rate on training tet:
0.0228603409678
**************
Error rate on test set: 
0.0726227106227
Accuracy rate on test set: 
0.927377289377
True positive rate on test set
0.0194863595483
True negative rate on test set
0.999201536844


In [17]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()

Error rate on training set: 
0.0615949764521
Accuracy rate on training set: 
0.938405023548
True positive rate on training tet:
0.571164112278
**************
Error rate on test set: 
0.115611721612
Accuracy rate on test set: 
0.884388278388
True positive rate on test set
0.280103927251
True negative rate on test set
0.932193876341


In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='linear', C=1.0, random_state=0)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_train)
y_pred_test = svm.predict(X_test)
print_results()